In [1]:
#install biopython into conda env
!conda install -c conda-forge biopython -y

Solving environment: done

## Package Plan ##

  environment location: /home/dylan/miniconda3/envs/qiime2-2021.2

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.78             |   py36h8f6f2f9_2         2.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.7 MB

The following packages will be UPDATED:

  biopython                             1.78-py36h8f6f2f9_1 --> 1.78-py36h8f6f2f9_2



biopython-1.78       | 2.7 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
from Bio import SeqIO
import os
from qiime2 import Artifact
from qiime2.plugins.feature_classifier.methods import extract_reads
from qiime2.plugins.feature_table.methods import merge_seqs, merge_taxa
import shutil
import tarfile
import tempfile
import urllib.request

In [3]:
def download_file(url, local_filepath):
    with urllib.request.urlopen(url) as response, open(local_filepath, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [2]:
working_dir = os.path.abspath('../')
refs_dir = working_dir + '/output/taxonomy_references'

In [5]:
if not os.path.exists(refs_dir):
    os.mkdir(refs_dir)
if not os.path.isfile(refs_dir + '/silva_sequences.qza'):
    download_file('https://data.qiime2.org/2021.2/common/silva-138-99-seqs-515-806.qza', 
                  refs_dir + '/silva_sequences.qza')
if not os.path.isfile(refs_dir + '/silva_taxonomy.qza'):
    download_file('https://data.qiime2.org/2021.2/common/silva-138-99-tax-515-806.qza', 
                  refs_dir + '/silva_taxonomy.qza')
if not os.path.isfile(refs_dir + '/gg_13_8_otus.tar.gz'):
    download_file('ftp://greengenes.microbio.me/greengenes_release/gg_13_5/gg_13_8_otus.tar.gz',
                  refs_dir + '/gg_13_8_otus.tar.gz')
if not os.path.isfile(refs_dir + '/Metaxa2_2.2.1.tar.gz'):
    download_file('https://microbiology.se/sw/Metaxa2_2.2.1.tar.gz',
                  refs_dir + '/Metaxa2_2.2.1.tar.gz')
if not os.path.isfile(refs_dir + '/PhytoRef_with_taxonomy.fasta'):
    download_file('http://phytoref.sb-roscoff.fr/static/downloads/PhytoRef_with_taxonomy.fasta',
                  refs_dir + '/PhytoRef_with_taxonomy.fasta')

In [6]:
with tarfile.open(refs_dir + '/gg_13_8_otus.tar.gz', 'r:gz') as tar:
    tar.extractall(refs_dir)
with tarfile.open(refs_dir + '/Metaxa2_2.2.1.tar.gz', 'r:gz') as tar:
    tar.extractall(refs_dir)

In [7]:
os.chdir(refs_dir + '/Metaxa2_2.2.1/metaxa2_db/SSU')
!blastdbcmd -entry all -db blast -out metaxa2.fasta
shutil.copyfile(refs_dir + '/Metaxa2_2.2.1/metaxa2_db/SSU/metaxa2.fasta', 
                refs_dir + '/metaxa2.fasta')
os.chdir(working_dir + '/procedure')

In [8]:
with open(refs_dir + '/silva_organelle_taxonomy.tsv', 'w') as silva_taxonomy:
    with open(refs_dir + '/gg_organelle_taxonomy.tsv', 'w') as gg_taxonomy:
        silva_taxonomy.write('Feature ID\tTaxon\n')
        gg_taxonomy.write('Feature ID\tTaxon\n')
        with open(refs_dir + '/organelle_sequences.fasta', 'w') as organelle_seqs:
            for i, entry in enumerate(SeqIO.parse(refs_dir + '/metaxa2.fasta', 'fasta')):
                if 'mitochondria' in entry.description or 'Mitochondria' in entry.description:
                    organelle_seqs.write('>metaxa2_mitochondria_' + str(i) + '\n')
                    organelle_seqs.write(str(entry.seq + '\n'))
                    specific_info = str(entry.description).split(';')[-1]
                    silva_taxonomy.write('metaxa2_mitochondria_' + str(i) + '\td__Bacteria; p__Proteobacteria; c__Alphaproteobacteria; o__Rickettsiales; f__Mitochondria; g__Mitochondria; s__' + specific_info + '\n')
                    gg_taxonomy.write('metaxa2_mitochondria_' + str(i) + '\tk__Bacteria; p__Proteobacteria; c__Alphaproteobacteria; o__Rickettsiales; f__mitochondria; g__Mitochondria; s__' + specific_info + '\n')
            for i, entry in enumerate(SeqIO.parse(refs_dir + '/PhytoRef_with_taxonomy.fasta', 'fasta')):
                if not 'XXXXXXXXXX' in entry.seq:   #ditch the weird sequence
                    organelle_seqs.write('>phytoref_chloroplast_' + str(i) + '\n')
                    organelle_seqs.write(str(entry.seq + '\n'))
                    specific_info = str(entry.description).split('|')[-1]
                    silva_taxonomy.write('phytoref_chloroplast_' + str(i) + '\td__Bacteria; p__Cyanobacteria; c__Cyanobacteriia; o__Chloroplast; f__Chloroplast; g__Chloroplast; s__' + specific_info + '\n')
                    gg_taxonomy.write('phytoref_chloroplast_' + str(i) + '\tk__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Chloroplast; f__Chloroplast; g__Chloroplast; s__' + specific_info + '\n')

In [3]:
#import, select V4 region, merge, save
organelle_seqs = Artifact.import_data('FeatureData[Sequence]',
                                      refs_dir + '/organelle_sequences.fasta')
v4_organelle_seqs, = extract_reads(organelle_seqs, 'GTGYCAGCMGCCGCGGTAA',
                                   'GGACTACNVGGGTWTCTAAT', n_jobs = 24,
                                   read_orientation = 'forward')
silva_extended_seqs, = merge_seqs([v4_organelle_seqs,
                                   Artifact.load(refs_dir +
                                                 '/silva_sequences.qza')])
silva_extended_seqs.save(refs_dir + '/silva_extended_sequences.qza')
gg_seqs = Artifact.import_data('FeatureData[Sequence]', refs_dir +
                               '/gg_13_8_otus/rep_set/99_otus.fasta')
#did trimming greengenes break everything?
v4_gg_seqs, = extract_reads(gg_seqs, 'GTGYCAGCMGCCGCGGTAA',
                            'GGACTACNVGGGTWTCTAAT', n_jobs = 24,
                            read_orientation = 'forward')
v4_gg_seqs.save(refs_dir + '/gg_sequences.qza')
gg_extended_seqs, = merge_seqs([organelle_seqs, gg_seqs])
gg_extended_seqs.save(refs_dir + '/gg_extended_sequences.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/gcmp/output/taxonomy_references/gg_extended_sequences.qza'

In [12]:
silva_organelle_taxonomy = Artifact.import_data('FeatureData[Taxonomy]',
                                                refs_dir +
                                                '/silva_organelle_taxonomy.tsv')
silva_extended_taxonomy, = merge_taxa([silva_organelle_taxonomy,
                                       Artifact.load(refs_dir +
                                                     '/silva_taxonomy.qza')])
silva_extended_taxonomy.save(refs_dir + '/silva_extended_taxonomy.qza')
gg_taxonomy = Artifact.import_data('FeatureData[Taxonomy]', refs_dir +
                                   '/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt',
                                   'HeaderlessTSVTaxonomyFormat')
gg_taxonomy.save(refs_dir + '/gg_taxonomy.qza')
gg_organelle_taxonomy = Artifact.import_data('FeatureData[Taxonomy]',
                                             refs_dir +
                                             '/gg_organelle_taxonomy.tsv')
gg_extended_taxonomy, = merge_taxa([gg_organelle_taxonomy, gg_taxonomy])
gg_extended_taxonomy.save(refs_dir + '/gg_extended_taxonomy.qza')

'/mnt/c/Users/dsone/Documents/zaneveld/gcmp/output/taxonomy_references/gg_extended_taxonomy.qza'